   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ------------------------- -------------- 7.3/11.5 MB 34.4 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 32.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------------------- ------------ 8.7/12.6 MB 40.6 MB/s eta 0:00:01
   ------------------------------- -------- 10.0/12.6 MB 27.7 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.6 MB 22.9 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 19.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np

In [6]:
from datetime import datetime, timedelta


In [7]:
num_records = 100  # Adjust this number as needed


# Example for Smart Logistics Tracking
data = []

In [8]:
for _ in range(num_records):
    record = {
        "timestamp": datetime.now() - timedelta(minutes=np.random.randint(0, 1440)),  # Random timestamp in the last 24 hours
        "rfid": f"TRK{np.random.randint(100, 999)}",  # Random RFID
        "gps_sensor": np.random.randint(60, 100),  # Normal heart rate range
        "temperature_sensor": round(np.random.uniform(20.0, 25.0), 1)  # Goods temperature in Celsius
    }
    data.append(record)

In [9]:
# Convert to DataFrame
df = pd.DataFrame(data)

In [10]:
print(df.temperature_sensor)

0     20.4
1     24.6
2     23.1
3     23.7
4     23.8
      ... 
95    24.9
96    22.9
97    22.8
98    24.9
99    24.0
Name: temperature_sensor, Length: 100, dtype: float64


In [11]:
df.to_csv("logistics_tracking_data.csv", index=False)
df.to_json("logistics_tracking_dat", orient="records")